In [1]:
import sys

# variables
charClass = 0; token = 0; 
lexeme = []; nextChar = ' '; nextToken = ' '; tokenString = ' '
index = 0; chars = []; perLine = []
idNum = 0; constNum = 0; opNum = 0; lineNum = 0
idArray = {}
constValue = 0
add_op = 0; mult_op = 0
grammar = 0; message = ' '
lastIndex = 0

#이번과제에 새로 정의한 global 변수
funcName = []; localvarName = []; num = 0; varName = []

# Character classes
LETTER = 0; DIGIT = 1; UNKNOWN = 2; EOF = 99

# Token codes
INT_LIT = 10; IDENT = 11; 
LEFT_BRACKET = 20; RIGHT_BRACKET = 21; SEMI_COLON = 22; COMMA = 23; 
VARIABLE = 30; CALL = 31; PRINT_ARI = 32;

#grammar correct
OK = 0; WARNING = 1; ERROR = 2

In [2]:
def getChar():
    # 다음 char를 nextChar에 저장하고 charClass 결정
    global nextChar, charClass, lastRow, lastColumn, index, lastIndex

    if index == lastIndex + 1:
        nextChar = "EOF"
    else:
        nextChar = chars[index]
    index = index + 1
    
    if nextChar == "EOF":
        charClass = EOF
    elif nextChar != ' ' and nextChar != "\n":
        if nextChar.isalpha() or nextChar == '_':
            charClass = LETTER
        elif nextChar.isdigit():
            charClass = DIGIT
        else:
            charClass = UNKNOWN
    else:
        charClass = EOF

In [3]:
def addChar():
    # lexeme 배열에 nextChar를 덧붙임
    global lexeme, nextChar
    lexeme.append(nextChar)

In [4]:
def lexical():
    # 식별자, 숫자, 연산자를 구별해서 nextToken에 셋 중에 어떤 것인지 저장
    global lexeme, charClass, nextToken, idNum, constNum, opNum, lineNum, perLine, tokenString, idArray, constValue, grammar, message
    del lexeme[:]
    getNonBlank()
    
    # 처음 char가 문자로 시작하면 identifier를 의미함, lexeme 문자열을 tokenString에 저장
    if charClass == LETTER:
        addChar(); getChar()
        while charClass == LETTER or charClass == DIGIT:
            addChar(); getChar()
        idNum = idNum + 1
        tokenString = ''.join(lexeme)
        if tokenString == 'variable':
            nextToken = VARIABLE
        elif tokenString == 'call':
            nextToken = CALL
        elif tokenString == 'print_ari':
            nextToken = PRINT_ARI
        else:
            nextToken = IDENT
    
    # 처음 char가 숫자로 시작하면 const를 의미함, lexeme 문자열을 constValue에 저장
#     elif charClass == DIGIT:
#         addChar(); getChar()
#         while charClass == DIGIT:
#             addChar(); getChar()
#         nextToken = INT_LIT
#         constNum = constNum + 1
#         constValue = int(''.join(lexeme))
    
    # 처음 char가 문자, 숫자가 아닌 special character로 시작하면 op를 의미함, lookup 함수 실행
    elif charClass == UNKNOWN:
        lookup(); getChar()
    
    # end of file
    elif charClass == EOF:
        nextToken = EOF
        lexeme.append('EOF0')
        if grammar == 0:
            print('(OK)')
        elif grammar == 1:
            print('(Warning)', message)
        elif grammar == 2:
            print('(Error)', message)

In [5]:
def lookup():
    # 문자와 숫자가 아닌 char를 받았을 때 operator(연산자) 구분
    global nextToken, nextChar, opNum, grammar, message
    if nextChar == '{':
        addChar(); nextToken = LEFT_BRACKET
    elif nextChar == '}':
        addChar(); nextToken = RIGHT_BRACKET
    elif nextChar == ',':
        addChar(); nextToken = COMMA
    elif nextChar == ';':
        addChar(); nextToken = SEMI_COLON
    else:
        addChar(); nextToken = EOF

In [6]:
def getNonBlank():
    # white-space 무시
    global nextChar
    while nextChar.isspace():
        getChar()

In [7]:
def start():
    # <start> → <functions>
    functions()
    
def functions():
    # <functions> → <function> { <function> }
    global nextToken, lineNum, perLine, grammar, message
    function()
    while nextToken != EOF:
        function()
        
def function():
    # <function> → <identifier> { <function_body> }
    global nextToken, tokenString, idArray, grammar, message, num, localvarName
    if nextToken == IDENT:
        lexical()
        funcName.append(tokenString)
        if nextToken == LEFT_BRACKET:
            lexical()
            print('함수 내부 첫 단어:', tokenString)
            function_body()
            if nextToken == RIGHT_BRACKET:
                lexical()
    varName.append(num)
    varName[num] = localvarName
    num += 1
    localvarName = []
    
def function_body():
    if nextToken == VARIABLE:
        var_definitions()
    statements();
    
def var_definitions():
    while nextToken == VARIABLE:
        var_definition()
    
def var_definition():
    lexical()
    print('variable 다음 변수:', tokenString)
    localvarName.append(tokenString)
    lexical()
    while nextToken == COMMA:
        lexical()
        localvarName.append(tokenString)
        lexical()
    if nextToken == SEMI_COLON:
        lexical()
    
#     if nextToken == VARIABLE:
#         lexical()
#         print('variable 다음 변수', tokenString)
#         varName = tokenString
#         print(varName)
#         lexical()
#         while nextToken == COMMA:
#             lexical()
#             varName = tokenString
#             print(varName)
#             lexical()
#         if nextToken == SEMI_COLON:
#             lexical()
#             # 변수명 받기
    
def statements():
    while nextToken != RIGHT_BRACKET:
        statement()
    
def statement():
    # <statement> → call <identifier>; |  print_ari; | <identifier>; 
    global nextToken, tokenString, constValue, expValue, grammar, message, opNum, nextChar, index
    if nextToken == CALL:
        lexical()
#         print('call 다음에', tokenString)
        lexical()
#         print('27이여야 정상:', nextToken)
        lexical()
        
    elif nextToken == PRINT_ARI:
        lexical()
        lexical()
    
    elif nextToken == IDENT:
        # 변수명 받기
#         print('statement 내부 변수 부르는 곳, p여야 정상:', tokenString)
        lexical()
#         print('27이여야 정상:', nextToken)
        lexical()
#         print('이제 }(26)이거 출력인가?', nextToken)
        

In [8]:
def warningMessage(code):
    if code == 1:
        message = '생략된 등호(=) 삽입'
    elif code == 2:
        message = '중복 연산자(+) 제거'
    elif code == 3:
        message = '중복 연산자(-) 제거'
    elif code == 4:
        message = '중복 연산자(-) 제거'
    elif code == 5:
        message = '중복 연산자(-) 제거'
    elif code == 6:
        message = '생략된 기호(:) 삽입'
    elif code == 7:
        message = '오른쪽 괄호가 없음'
        
    return message

In [9]:
def errorMessage(code):
    if code == 1:
        message = '정의되지 않은 변수가 참조됨'
    elif code == 2:
        message = '식별자(identifier) 뒤에 등호(=)가 없음'
    elif code == 3:
        message = '문장이 식별자로 시작하지 않음'

    return message

In [10]:
def has_duplicates(seq):
    return len(seq) != len(set(seq))

In [11]:
try:
#     with open(sys.argv[1], "r") as file:
    with open('txt1.txt', "r") as file:
        list_line = file.read()
        for line in range(0, len(list_line)):
            chars.append(list_line[line])
    
    lastIndex = len(list_line) - 1
    lineNum = len(list_line.split('\n'))
    
    for per in range(0, lineNum):
        perLine.append(list_line.split('\n')[per])
    lineNum = 0

    getChar()
    lexical()
    start()
    
    print('함수들')
    print(funcName)
    print('변수들')
    for i in range(num):
        print(varName[i])
        
    if has_duplicates(funcName):
        print('함수에 중복이름 있다')
    else:
        print('함수에 중복이름 없다')
        
    if 'main' not in funcName:
        print('main 없다')
    else:
        print('main 있다')
    
        
except Exception as e:
    print('ERROR - ', e)
    sys.exit()

함수 내부 첫 단어: variable
variable 다음 변수: a
함수 내부 첫 단어: variable
variable 다음 변수: x
함수 내부 첫 단어: variable
variable 다음 변수: p
(OK)
함수들
['first', 'second', 'main']
변수들
['a', 'b', 'c']
['x', 'a']
['p', 'q']
함수에 중복이름 없다
main 있다


In [12]:
funcName=[]
num = 0
funcName.append('first')

ar=[]

ar.append(num)
ar[num] = list()
ar[num].append('Local variable: p')
ar[num].append('Local variable: q')

funcName.append('second')
num += 1
ar.append(num)
ar[num] = list()
ar[num].append('Return Address: main:1')
ar[num].append('Dynamic Link: 1')

ar[1]

['Return Address: main:1', 'Dynamic Link: 1']

In [13]:
a = [1, 2, 3]
c = [10, 11, 12]
b = []
num = 0
b.append('first')
b[num] = a
b[num]
num += 1
b.append('sec')
b[num] = c
b[num]
b[1]

[10, 11, 12]